# Classical Molecular Interaction Potentials tutorial using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of computing **classical molecular interaction potentials** from **protein structures**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **XXXX** protein (PDB code [XXXX](https://www.rcsb.org/structure/XXXX)).   
***

## Settings

### Biobb modules used

 - [biobb_cmip](https://github.com/bioexcel/biobb_cmip): Tools to compute classical molecular interaction potentials from protein structures.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [plotly](https://plotly.com/python/): Python Open Source Graphing Library. 


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_cmip.git
cd biobb_wf_cmip
conda env create -f conda_env/environment.yml
conda activate biobb_wf_cmip
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_cmip/notebooks/biobb_wf_cmip.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [PDB preparation (from PDB databank)](#preparePDB)
 3. [Molecular Interaction Potentials](#mips)
    1. [MIP+](#mip_pos) 
    2. [MIP-](#mip_neg) 
    3. [MIPn](#mip_neutral) 
 4. [PDB preparation (from MD)](#preparePDB_MD)
 5. [Interaction Potentials](#interaction)
    1. [Ligand](#ligand)
    2. [Protein](#protein)
 6. [Titration](#titration)
 7. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [9]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

data_folder = "data/"
pdbCode = "1aki"
inputPDB = data_folder + pdbCode + ".pdb"

MDCode = "RBD-hACE2"
inputPDB_MD = data_folder + MDCode + ".noZN.pdb" 
inputTOP_MD = data_folder + MDCode + ".top" 

<a id="preparePDB"></a>
***
## PDB Preparation (from PDB databank)
CMIP tool needs additional information (e.g. charges, elements) included in the structure PDB file to properly run. In the next cells, a couple of BioBB building blocks will be used to prepare the input PDB file, adding this extra information.   
***
**Building Blocks** used:
 - [prepare_pdb](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_pdb) from **biobb_cmip.cmip.prepare_pdb**
***

In [10]:
from biobb_cmip.cmip.prepare_pdb import prepare_pdb

cmipPDB = pdbCode + ".cmip.pdb"

prepare_pdb(input_pdb_path=inputPDB,
            output_cmip_pdb_path=cmipPDB)

2022-03-17 12:16:34,087 [MainThread  ] [INFO ]  Not using any container
2022-03-17 12:16:34,088 [MainThread  ] [INFO ]  check_structure -v -i data/1aki.pdb -o 1aki.cmip.pdb --output_format cmip --non_interactive command_list --list 'water --remove yes; backbone --add_caps none; fixside --fix All; add_hydrogen --add_mode auto --add_charges CMIP'

2022-03-17 12:16:34,852 [MainThread  ] [INFO ]  Exit code 0

2022-03-17 12:16:34,854 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.10                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure data/1aki.pdb loaded
 Title: the structure of the orthorhombic form of hen egg-white lysozyme at 1.5 angstroms resolution
 Experimental method: x-ray diffraction
 Keywords: hydrolase, glycosidase
 Resolution (A): 1.5

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  207
 Num. r

0

<a id="mips"></a>
***
## Molecular Interaction Potentials (MIPs)
...   
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

In [11]:
from biobb_cmip.cmip.cmip import cmip

mip_pos_log = pdbCode + ".mip_pos.log"
mip_pos_cube = pdbCode + ".mip_pos.cube"

prop = { 
    'execution_type' : 'mip_pos',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_pos_log,
     output_cube_path=mip_pos_cube,
     properties=prop)

2022-03-17 12:16:38,445 [MainThread  ] [INFO ]  Copy: 1aki.cmip.pdb to /Users/pau/projects/biobb_cmip/jupyter_notebook/86a4aeab-2d6c-4977-ba87-7b79af59480c
2022-03-17 12:16:38,448 [MainThread  ] [INFO ]  Copy: /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm to /Users/pau/projects/biobb_cmip/jupyter_notebook/86a4aeab-2d6c-4977-ba87-7b79af59480c
2022-03-17 12:16:38,450 [MainThread  ] [INFO ]  Copy: 39447092-b1df-4fcf-9667-70faf2c9c19e/params to /Users/pau/projects/biobb_cmip/jupyter_notebook/86a4aeab-2d6c-4977-ba87-7b79af59480c
2022-03-17 12:16:38,451 [MainThread  ] [INFO ]  Using Docker image quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0
2022-03-17 12:16:38,452 [MainThread  ] [INFO ]  docker run -v /Users/pau/projects/biobb_cmip/jupyter_notebook/86a4aeab-2d6c-4977-ba87-7b79af59480c:/inout quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0 /bin/bash -c "cmip -i /inout/params -vdw /inout/vdwprm -hs /inout/1aki.cmip.pdb -cube /inout/1aki.mip_pos.cube -o /inout/1aki.mip_pos.log"

2022-03-17 

0

In [12]:
! cat 1aki.mip_pos.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   11:16:39 h on 17- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: /inout/params                                                                                                                                         
 VdW Parameters:      /inout/vdwprm                                                                                                                                         
 PDB Host:            /inout/1aki.cmip.pdb                                                                                              

In [13]:
view = nglview.show_structure_file(mip_pos_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="blue")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [14]:
from biobb_cmip.cmip.cmip import cmip

mip_neg_log = pdbCode + ".mip_neg.log"
mip_neg_cube = pdbCode + ".mip_neg.cube"

prop = { 
    'execution_type' : 'mip_neg',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_neg_log,
     output_cube_path=mip_neg_cube,
     properties=prop)

2022-03-17 12:16:55,594 [MainThread  ] [INFO ]  Copy: 1aki.cmip.pdb to /Users/pau/projects/biobb_cmip/jupyter_notebook/85c46788-eac8-436b-b3cc-d354bbc37317
2022-03-17 12:16:55,596 [MainThread  ] [INFO ]  Copy: /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm to /Users/pau/projects/biobb_cmip/jupyter_notebook/85c46788-eac8-436b-b3cc-d354bbc37317
2022-03-17 12:16:55,599 [MainThread  ] [INFO ]  Copy: fb02afc2-e4af-4b1c-a250-814bbf22598b/params to /Users/pau/projects/biobb_cmip/jupyter_notebook/85c46788-eac8-436b-b3cc-d354bbc37317
2022-03-17 12:16:55,600 [MainThread  ] [INFO ]  Using Docker image quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0
2022-03-17 12:16:55,601 [MainThread  ] [INFO ]  docker run -v /Users/pau/projects/biobb_cmip/jupyter_notebook/85c46788-eac8-436b-b3cc-d354bbc37317:/inout quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0 /bin/bash -c "cmip -i /inout/params -vdw /inout/vdwprm -hs /inout/1aki.cmip.pdb -cube /inout/1aki.mip_neg.cube -o /inout/1aki.mip_neg.log"

2022-03-17 

0

In [15]:
! cat 1aki.mip_neg.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   11:16:56 h on 17- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: /inout/params                                                                                                                                         
 VdW Parameters:      /inout/vdwprm                                                                                                                                         
 PDB Host:            /inout/1aki.cmip.pdb                                                                                              

In [16]:
view = nglview.show_structure_file(mip_neg_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="red")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [17]:
from biobb_cmip.cmip.cmip import cmip

mip_neutral_log = pdbCode + ".mip_neutral.log"
mip_neutral_cube = pdbCode + ".mip_neutral.cube"

prop = { 
    'execution_type' : 'mip_neu',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_neutral_log,
     output_cube_path=mip_neutral_cube,
     properties=prop)

2022-03-17 12:17:16,199 [MainThread  ] [INFO ]  Copy: 1aki.cmip.pdb to /Users/pau/projects/biobb_cmip/jupyter_notebook/d17355a7-febf-46aa-a1fd-f0cf6aa8a6fe
2022-03-17 12:17:16,202 [MainThread  ] [INFO ]  Copy: /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm to /Users/pau/projects/biobb_cmip/jupyter_notebook/d17355a7-febf-46aa-a1fd-f0cf6aa8a6fe
2022-03-17 12:17:16,205 [MainThread  ] [INFO ]  Copy: e376c5dd-3c9b-45fa-878a-d3265cb61be6/params to /Users/pau/projects/biobb_cmip/jupyter_notebook/d17355a7-febf-46aa-a1fd-f0cf6aa8a6fe
2022-03-17 12:17:16,206 [MainThread  ] [INFO ]  Using Docker image quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0
2022-03-17 12:17:16,207 [MainThread  ] [INFO ]  docker run -v /Users/pau/projects/biobb_cmip/jupyter_notebook/d17355a7-febf-46aa-a1fd-f0cf6aa8a6fe:/inout quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0 /bin/bash -c "cmip -i /inout/params -vdw /inout/vdwprm -hs /inout/1aki.cmip.pdb -cube /inout/1aki.mip_neutral.cube -o /inout/1aki.mip_neutral.log"

202

0

In [18]:
view = nglview.show_structure_file(mip_neutral_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-1, color="grey")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [19]:
#Show different structures generated (for comparison)
view1 = nglview.show_structure_file(cmipPDB)
view1.add_component(mip_pos_cube)
view1.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view1.component_1.add_surface(isolevelType="value", isolevel=-5, color="blue")
view1.component_0.center()
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(cmipPDB)
view2.add_component(mip_neg_cube)
view2.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view2.component_1.add_surface(isolevelType="value", isolevel=-5, color="red")
view2.component_0.center()
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(cmipPDB)
view3.add_component(mip_neutral_cube)
view3.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view3.component_1.add_surface(isolevelType="value", isolevel=-1, color="grey")
view3.component_0.center()
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="preparePDB_MD"></a>
***
## PDB Preparation (from Molecular Dynamics)
...
***
**Building Blocks** used:
 - [prepare_structure](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_structure) from **biobb_cmip.cmip.prepare_structure**
***

In [20]:
from biobb_structure_utils.utils.extract_chain import extract_chain

input_extract_chain = data_folder + "md.imaged.rot.dry.pdb"
inputPDB_MD_hACE2 = MDCode + ".hACE2.pdb"
inputPDB_MD_RBD = MDCode + ".RBD.pdb"

prop = {
    'chains': [ 'A' ],
    'permissive': True
}
extract_chain(input_structure_path=input_extract_chain,
              output_structure_path=inputPDB_MD_hACE2,
              properties=prop)

prop = {
    'chains': [ 'B' ],
    'permissive': True
}
extract_chain(input_structure_path=inputPDB_MD,
              output_structure_path=inputPDB_MD_RBD,
              properties=prop)

2022-03-17 12:17:29,606 [MainThread  ] [INFO ]  Selected Chains: A
2022-03-17 12:17:29,607 [MainThread  ] [INFO ]  Warning: Use permissive=True is a risky option use it under your own responsability
2022-03-17 12:17:29,643 [MainThread  ] [INFO ]  Removed: []
2022-03-17 12:17:29,647 [MainThread  ] [INFO ]  Selected Chains: B
2022-03-17 12:17:29,647 [MainThread  ] [INFO ]  Warning: Use permissive=True is a risky option use it under your own responsability
2022-03-17 12:17:29,686 [MainThread  ] [INFO ]  Removed: []


In [21]:
from biobb_cmip.cmip.prepare_structure import prepare_structure

cmipPDB_MD_hACE2 = MDCode + ".hACE2.cmip.pdb"
cmipPDB_MD_RBD = MDCode + ".RBD.cmip.pdb"

prepare_structure(input_pdb_path=inputPDB_MD_hACE2,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=cmipPDB_MD_hACE2)

prepare_structure(input_pdb_path=inputPDB_MD_RBD,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=cmipPDB_MD_RBD)

2022-03-17 12:17:29,700 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract charges
2022-03-17 12:18:14,169 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract elements
2022-03-17 12:19:24,153 [MainThread  ] [INFO ]  Removed: []
2022-03-17 12:19:24,161 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract charges
2022-03-17 12:20:06,336 [MainThread  ] [INFO ]  Reading: data/RBD-hACE2.top to extract elements
2022-03-17 12:21:05,359 [MainThread  ] [INFO ]  Removed: []


0

<a id="interaction"></a>
***
## Interaction Potential Energies 
... 
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

In [22]:
from biobb_cmip.cmip.cmip import cmip

######### Remove this #############
cmipPDB_MD_hACE2 = "RBD-hACE2" + ".hACE2.cmip.pdb"
cmipPDB_MD_RBD   = "RBD-hACE2" + ".RBD.cmip.pdb"
data_folder = "data/"
pdbCode = "1aki"
inputPDB = data_folder + pdbCode + ".pdb"

MDCode = "RBD-hACE2"
inputPDB_MD = data_folder + MDCode + ".noZN.pdb" 
inputTOP_MD = data_folder + MDCode + ".top" 
###################################

RBD_energies_log = "RBD.energies.log"
RBD_byat_out = "RBD.energies.byat.out"

prop = { 
    'execution_type' : 'energy',
    'remove_tmp': False
}

cmip(input_pdb_path=cmipPDB_MD_hACE2,
     input_probe_pdb_path=cmipPDB_MD_RBD,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     properties=prop)

2022-03-17 12:21:05,378 [MainThread  ] [INFO ]  Not using any container
2022-03-17 12:21:05,379 [MainThread  ] [INFO ]  cmip -i d45a16b5-9c50-49ed-96e8-bd2a7f47485b/params -vdw /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm -hs RBD-hACE2.hACE2.cmip.pdb -pr RBD-hACE2.RBD.cmip.pdb -byat RBD.energies.byat.out -o RBD.energies.log

2022-03-17 12:21:24,733 [MainThread  ] [INFO ]  Exit code 0



0

In [6]:
!cat RBD.energies.log

 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at   12:15:19 h on 17- 3-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: db90dc9d-59b8-410d-b45b-29349acb97a3/params                                                                                                           
 VdW Parameters:      /Users/pau/anaconda3/envs/biobb/share/cmip/dat/vdwprm                                                                                                 
 PDB Host:            RBD-hACE2.hACE2.cmip.pdb                                                                                          

In [ ]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(RBD_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat

atom_list, energy_dict = get_energies_byat(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [ ]:
from biobb_cmip.cmip.prepare_structure import prepare_structure


box_pdb_output = 'output_tit_all.pdb'

prepare_structure(input_pdb_path=inputPDB_MD,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=box_pdb_output)


In [ ]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "hACE2.box.log"
RBD_byat_out = "hACE2.box.byat.out"
json_box_out = "box.json"

prop = { 
    'execution_type' : 'check_only',
    'remove_tmp':False,
    'box_size_factor': 0.9
}

cmip(input_pdb_path=box_pdb_output,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     output_json_box_path=json_box_out,
     properties=prop)

In [ ]:
! cat hACE2.box.log

In [ ]:
! cat box.json

In [ ]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(json_box_out, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view



In [ ]:
!cat output_tit_all.pdb

In [ ]:
%load_ext autoreload
%autoreload 2

from biobb_cmip.cmip.ignore_residues import ignore_residues


inputPDB_MD_hACE2_ignored = "inputPDB_MD_hACE2_ignored.pdb"




prop = {
    'residue_list': "A:"
}

ignore_residues(input_cmip_pdb_path = box_pdb_output,
               output_cmip_pdb_path = inputPDB_MD_hACE2_ignored,
               properties = prop)

In [ ]:
! cat inputPDB_MD_hACE2_ignored.pdb

In [ ]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "hACE2.energies.log"
RBD_byat_out = "hACE2.energies.byat.out"

prop = { 
    'execution_type' : 'energy',
    'remove_tmp':False,
}

cmip(input_pdb_path=inputPDB_MD_hACE2_ignored,
     input_probe_pdb_path=cmipPDB_MD_hACE2,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     input_json_box_path=json_box_out,
     properties=prop)

In [ ]:
! cat hACE2.energies.log

In [ ]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(RBD_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat


atom_list, energy_dict = get_energies_byat(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Interaction Energy Kcal/mol"))}

plotly.offline.iplot(fig)



<a id="titration"></a>
***
## PDB Titration
...
***
**Building Blocks** used:
 - [titration](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.titration) from **biobb_cmip.cmip.titration**
 - [cat_pdb](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

In [ ]:
from biobb_cmip.cmip.titration import titration

titration_input_pdb = data_folder + 'input2.pdb'
titration_output_pdb = 'output_tit.pdb'
titration_output_log = 'output_tit.log' 

prop = { 
    'num_positive_ions' : 5,
    'num_negative_ions' : 5,    
    'num_wats' : 20,
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

titration(input_pdb_path=titration_input_pdb,
          output_pdb_path=titration_output_pdb,
          output_log_path=titration_output_log,
          properties=prop)

In [ ]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

cat_pdb_output = 'output_tit_all.pdb'

cat_pdb(input_structure1=titration_input_pdb,
        input_structure2=titration_output_pdb,
        output_structure_path=cat_pdb_output)

In [ ]:
view = nglview.show_structure_file(cat_pdb_output)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='spacefill', selection='water')
view.add_representation(repr_type='spacefill', selection='.Na', color='element')
view.add_representation(repr_type='spacefill', selection='.Cl', color='element')
view._remote_call('setSize', target='Widget', args=['','600px'])
view